# Zambon - List

## Wichtig
* PDF-Konvertierung mit Tabula kommt immer schräg raus. Daher wird das PDF in eine HTML-Tabelle konvertiert. Dazu müssen aber händisch einige Schritte vorgenommen werden. Script folgen.

In [36]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import copy
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## PDF to HTML
Dieser Schritt muss **manuell** getätigt werden!
* PDF `WEB Zambon CH 2018 (Stand am 20.06.2019)_0.pdf` in Adobe Acrobat (nicht Reader!) öffnen
* Datei -> Exportieren in -> HTML Website
* Speichern als `website_dump.html`

## Import HTML

In [89]:
df_list = pd.read_html("website_dump.html", "",  parse_dates=False, thousands='~')
df = pd.concat(df_list)

## Format Table

In [90]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total', 'empty2', 'empty3']

#Replace by string
df_export.loc[df_export['location'] == 'HCPs', 'location'] = np.nan
df_export.loc[df_export['location'] == 'HCOs', 'location'] = np.nan

#Shift
df_export['empty1'] = df_export['empty1'].fillna("")
df_export['empty2'] = df_export['empty2'].fillna("")
df_export['empty3'] = df_export['empty3'].fillna("")
df_export['empty1'] = df_export['empty1'].astype(str)
df_export['empty2'] = df_export['empty2'].astype(str)
df_export['empty3'] = df_export['empty3'].astype(str)
df_export[df_export['empty3'] != ""] = df_export[df_export['empty3'] != ""].shift(-2, axis='columns')

#Add Type
df_export = df_export.reset_index(drop=True)
index_hco = df_export[df_export['name'].str.contains("OTHER, NOT INCLUDED", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty rows
df_export = df_export.dropna(subset=['name'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')

#Remove empty
df_export.drop(columns=['empty1', 'empty2', 'empty3'], inplace=True)

#Reomve by str
df_export = df_export[~df_export['name'].str.contains("R&D")]

#Convert to Numbers
df_export = replace_comma_to_dot(df_export)
df_export = amounts_to_number(df_export)

#Reorder columns
df_export = add_plz(df_export)
df_export = df_export[fix_columns[:-1]]

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'zambon')

Total nicht Summe der Werte
saved


In [ ]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)